In [41]:
import json

In [42]:
syms = []
symfile = open('../snp500sym.csv','r')
symlines = symfile.readlines()
for line in symlines:
    syms.append(line.strip())
print(syms[:5])
print(f'no of stocks = {len(syms)}')

['MMM', 'AOS', 'ABT', 'ABBV', 'ABMD']
no of stocks = 505


In [43]:
bal = open('INCOME_STATEMENT/AAL.json', 'r')
inc = open('BALANCE_SHEET/AAL.json', 'r')
bal = json.load(bal)
inc = json.load(inc)
inc = inc['annualReports'][0]
bal = bal['annualReports'][0]
income_statement_indicators = [u[0] for u in inc.items()]
balance_sheet_indicators = [u[0] for u in bal.items()]
income_statement_indicators = set(income_statement_indicators)
balance_sheet_indicators = set(balance_sheet_indicators)
# print(income_statement_indicators)
# print(balance_sheet_indicators)
indicators = income_statement_indicators.union(balance_sheet_indicators)
print(indicators)
# print(len(indicators))


{'researchAndDevelopment', 'operatingExpenses', 'fiscalDateEnding', 'accumulatedDepreciationAmortizationPPE', 'shortTermInvestments', 'commonStockSharesOutstanding', 'inventory', 'totalCurrentLiabilities', 'totalRevenue', 'totalShareholderEquity', 'totalNonCurrentAssets', 'grossProfit', 'shortTermDebt', 'currentDebt', 'incomeTaxExpense', 'currentLongTermDebt', 'otherCurrentLiabilities', 'propertyPlantEquipment', 'nonInterestIncome', 'investments', 'totalNonCurrentLiabilities', 'longTermInvestments', 'ebitda', 'netInterestIncome', 'deferredRevenue', 'incomeBeforeTax', 'cashAndCashEquivalentsAtCarryingValue', 'longTermDebtNoncurrent', 'ebit', 'intangibleAssetsExcludingGoodwill', 'cashAndShortTermInvestments', 'reportedCurrency', 'costofGoodsAndServicesSold', 'otherNonOperatingIncome', 'otherCurrentAssets', 'interestIncome', 'otherNonCurrrentAssets', 'capitalLeaseObligations', 'goodwill', 'commonStock', 'depreciationAndAmortization', 'depreciation', 'sellingGeneralAndAdministrative', 'cos

In [44]:
period_file = open(f'../data/QuarterlyData/2021_Q4.csv', 'r')
indicators = period_file.readline()
lines = period_file.readlines()
ticker = []
for line in lines:
    line = line.strip().split(',')
    ticker.append(line[0])
indicators = indicators.strip().split(',')
# print(indicators)
indicatos_ind = dict()
for i in range(len(indicators)):
    indicatos_ind[indicators[i].lower()] = i+1
print(indicatos_ind)

{'ticker': 1, 'reportedcurrency': 2, 'totalassets': 3, 'totalcurrentassets': 4, 'cashandcashequivalentsatcarryingvalue': 5, 'cashandshortterminvestments': 6, 'inventory': 7, 'currentnetreceivables': 8, 'totalnoncurrentassets': 9, 'propertyplantequipment': 10, 'accumulateddepreciationamortizationppe': 11, 'intangibleassets': 12, 'intangibleassetsexcludinggoodwill': 13, 'goodwill': 14, 'investments': 15, 'longterminvestments': 16, 'shortterminvestments': 17, 'othercurrentassets': 18, 'othernoncurrrentassets': 19, 'totalliabilities': 20, 'totalcurrentliabilities': 21, 'currentaccountspayable': 22, 'deferredrevenue': 23, 'currentdebt': 24, 'shorttermdebt': 25, 'totalnoncurrentliabilities': 26, 'capitalleaseobligations': 27, 'longtermdebt': 28, 'currentlongtermdebt': 29, 'longtermdebtnoncurrent': 30, 'shortlongtermdebttotal': 31, 'othercurrentliabilities': 32, 'othernoncurrentliabilities': 33, 'totalshareholderequity': 34, 'treasurystock': 35, 'retainedearnings': 36, 'commonstock': 37, 'com

In [60]:
ratios = [
    # ['pe', [], []], 
    # ['eps', [], []],
    # ['de', ['totalLiabilities'], ['totalShareHolderEquity']], 
    ['da', ['totalLiabilities'], ['totalAssets']],
    # ['roe', ['netincome'], ['totalShareHolderEquity']],
    ['roa', ['netincome'], ['totalAssets']],
    ['net_profit_margin', ['netincome'], ['totalrevenue']],
    # ['current_ratio', ['totalCurrentAssets'], ['totalCurrentLiabilities']],
    # ['quick_ratio', ['totalcurrentassets', '-inventory'], ['totalcurrentliabilities']],
    ['asset_turnover', ['totalrevenue'], ['totalAssets']],
    ['debt_ratio', ['totalliabilities'], ['totalassets']],
]

In [61]:
for year in range(2017, 2022):
    for qtr in range(1, 5):
        period = f'{year}_Q{qtr}'
        period_file = open(f'../data/QuarterlyData/{period}.csv', 'r')
        header = period_file.readline()
        lines = period_file.readlines()
        info = []
        for line in lines:
            line = line.strip()
            line = line.split(',')
            info.append(line)
        # print(info[:5])
        period_ratios = open(f'../data/QuarterlyRatios/{period}.csv', 'w')
        ratio_header = ['ticker']
        for ratio in ratios:
            ratio_header.append(ratio[0])
        period_ratios.write(','.join(ratio_header)  + '\n')
        for i in range(len(ticker)):
            to_write = [ticker[i]]
            for ratio in ratios:
                
                num = 0
                den = 0
                    
                try:
                    for nums in ratio[1]:
                        if (nums.lower())[0] == '-':
                            num -= float(info[i][indicatos_ind[(nums.lower())[1:]]])
                        else:
                            num += float(info[i][indicatos_ind[nums.lower()]])
                    for dens in ratio[2]:
                        if (dens.lower())[0] == '-':
                            den -= float(info[i][indicatos_ind[(dens.lower())[1:]]])
                        else:
                            den += float(info[i][indicatos_ind[dens.lower()]])
                    to_write.append(str(num/den))
                    pass
                except:
                    if num != 0:
                        # print(num, den)
                        pass
                    to_write.append('#####')
                    pass
            
            period_ratios.write(','.join(to_write) + '\n')

In [64]:
tickerset = set(ticker)
badtickerset = set()
year_start = 2017
year_end = 2019
for year in range(year_start, year_end):
    for qtr in range(1, 5):
        period = f'{year}_Q{qtr}'
        period_file = open(f'../data/QuarterlyRatios/{period}.csv', 'r')
        header = period_file.readline()
        lines = period_file.readlines()
        period_file.close()
        cool = 0
        for line in lines:
            ok = True
            line = line.strip().split(',')
            for word in line:
                if word == '#####':
                    ok = False
                    break
            if not ok:
                if tickerset.__contains__(line[0]):
                    tickerset.remove(line[0])
                badtickerset.add(line[0])
            else:
                cool += 1
        print(cool,end=' ')
print()
print(len(tickerset), tickerset)
print(len(badtickerset), badtickerset)

471 476 479 471 478 482 483 488 
428 {'PEAK', 'LYB', 'EQR', 'CTAS', 'VNO', 'FRT', 'EXPE', 'CMG', 'CTRA', 'UNH', 'TGT', 'MET', 'L', 'SYY', 'GM', 'SNPS', 'AVB', 'PNW', 'NVR', 'AEE', 'NWS', 'PEG', 'STZ', 'TDG', 'GPC', 'ABC', 'RSG', 'ALGN', 'GWW', 'LOW', 'CNP', 'ANTM', 'IT', 'XRAY', 'IR', 'UA', 'TTWO', 'WDC', 'FTV', 'RJF', 'FDS', 'ODFL', 'CVS', 'DAL', 'NDSN', 'NDAQ', 'DFS', 'LRCX', 'CF', 'AIG', 'MPWR', 'ROP', 'WRB', 'CHRW', 'REGN', 'CBRE', 'ROST', 'LLY', 'IEX', 'PKG', 'NKE', 'JBHT', 'OGN', 'WAT', 'KMB', 'PTC', 'CMCSA', 'NLSN', 'NOW', 'EMN', 'SBUX', 'QCOM', 'SYF', 'PCAR', 'MMM', 'NSC', 'ESS', 'DRE', 'MTB', 'GPN', 'PPL', 'SBNY', 'MS', 'NEM', 'MA', 'WBA', 'AJG', 'FFIV', 'STT', 'FAST', 'ABT', 'CBOE', 'USB', 'ALLE', 'FBHS', 'BK', 'BRO', 'GL', 'XEL', 'COF', 'GE', 'CSCO', 'TDY', 'WTW', 'MPC', 'AMT', 'HII', 'D', 'NTAP', 'WFC', 'HPE', 'WHR', 'PHM', 'ITW', 'DGX', 'LEN', 'ROL', 'TECH', 'LYV', 'TRV', 'CEG', 'POOL', 'HLT', 'DVA', 'F', 'MMC', 'URI', 'GLW', 'AAL', 'PEP', 'MCD', 'MNST', 'TYL', 'NLOK', 'UN

In [65]:
for year in range(year_start, year_end):
    for qtr in range(1, 5):
        period = f'{year}_Q{qtr}'
        period_file = open(f'../data/QuarterlyRatios/{period}.csv', 'r')
        header = period_file.readline()
        lines = period_file.readlines()
        period_file.close()
        period_file = open(f'../data/QuarterlyRatiosClean/{period}.csv', 'w')
        period_file.write(header)
        for line in lines:
            line = line.strip().split(',')
            if tickerset.__contains__(line[0]):
                period_file.write(line[0] + ',' + ','.join(line[1:]) + '\n')